In [1]:
class Inode:
    def __init__(self, nome, tamanho, isDir = False):
        #comum entre arq e dir
        self.nome = nome
        self.tam = tamanho
        self.isDir = isDir
        self.pai = None
        #somente arq
        self.dados = []
        #somente dir
        self.filhos = {}

class arquivos:
    def __init__(self):
        self.root = Inode("/", 0, True)
        self.aux = self.root
   
    #mk
    def criaArquivo(self, nome):
        if nome in self.aux.filhos:
            print(f"arquivo com nome '{nome}' já existente no diretório atual")
            return
        #arquivo q não tem extensão vira .txt
        if len(nome.split(".")) == 1:
          nome = nome + ".txt"

        arq = Inode(nome, 0)
        arq.pai = self.aux
        self.aux.filhos[nome] = arq
   
    #mkdir
    def criaDiretorio(self, nome):
        if nome in self.aux.filhos:
            print(f"diretório com nome '{nome}' já existente no diretório atual")
            return
        dir = Inode(nome, 0, True)
        dir.pai = self.aux
        self.aux.filhos[nome] = dir            

    #cd
    def moverParaDiretorio(self, nome):
       
        nomeArquivo = nome.split("/")
        if len(nomeArquivo) > 1 and nome != "/":
            self.aux = self.pathAbsoluto(nome)
            nome = nomeArquivo[-1]
        if nome == "/":
            self.aux = self.root
            return
        if nome == ".":
            return
        if nome == ".." and self.aux.pai != None:
            self.aux = self.aux.pai
            return
        if nome != "..":
            if nome not in self.aux.filhos:
                print(f"diretório {nome} não encontrado")
                return
            if not self.aux.filhos[nome].isDir:
                print(f"não é possível mover para um arquivo")
                return
            self.aux = self.aux.filhos[nome]

    #wa
    def escreverNoArquivo(self, nome, dados):
        if nome not in self.aux.filhos:
            print(f"arquivo '{nome}' não encontrado no diretório atual")
            return
        node:Inode = self.aux.filhos[nome]
        if node.isDir:
            print(f"item de nome '{nome}' é referente a um dicionário")
            return
        node.dados.append(dados)
        node.tam += len(dados)

    #ra 
    def lerArquivo(self, nome):
        if nome not in self.aux.filhos:
            print(f"arquivo '{nome}' não encontrado no diretório atual")
            return
        print(f"\nLendo arquivo '{nome}'\n")
        node:Inode = self.aux.filhos[nome]
        if node.isDir:
            print(f"item de nome '{nome}' é referente a um dicionário")
            return
        for dado in node.dados:
            print("\t", dado)
        print(f"\n\ttamanho do arquivo: {node.tam}")

    #dir
    def listarItensDirAtual(self):
        nomeDir = self.obterPathAtual()
        print(f"\nDiretório {nomeDir}\n")
        for nodeName, node in self.aux.filhos.items():
            tipo = "DIR" if node.isDir else "ARQ"
            print(f"\t{tipo}  {nodeName}")
        print()

    #rm
    def removerItens(self, nome, param = ""):
        try:
            root = self.aux
            nomeArquivo = nome.split("/")
            if len(nomeArquivo) > 1:
                self.pathAbsoluto(nome)
                if root.filhos[nomeArquivo[-1]].filhos != {} and param != "-f":
                    print("Não é possível apagar diretórios com itens dentro")
                    return
                del root.filhos[nomeArquivo[-1]]
            else:
                if root.filhos[nome].filhos != {} and param != "-f":
                    print("Não é possível apagar diretórios com itens dentro")
                    return
                del root.filhos[nome]
        except KeyError:
            print(f"arquivo '{nome}' não encontrado")

    #retorna o diretório absoluto de destino
    def pathAbsoluto(self, destino):
        destinoSplit = destino.split("/")
        backup = self.aux
        curr = self.aux
        self.aux = self.root
        while destinoSplit:
            inode = destinoSplit.pop(0)
            if inode != "." and inode != ".." and inode.__contains__("."):
                root = self.aux.filhos[inode if inode != "" and len(inode.split(".")) > 1 else destinoSplit.pop(0)]
            elif inode == "..":
                self.aux = curr
                root = self.aux.pai
                curr = root
            elif inode == ".":
                self.aux = curr
                root = self.aux
                curr = root
            else:
                root = self.aux.filhos[inode if inode != "" else destinoSplit.pop(0)]
            self.aux = root
        self.aux = backup
        if root.pai != None:
            return root.pai
        return root

    #retorna o path atual
    def obterPathAtual(self):
        backup = self.aux
        nomeDir = []
        while self.aux.pai:
            nomeDir.append(self.aux.nome)
            self.aux = self.aux.pai
        self.aux = backup
        nomeDir = "/"+"/".join(nomeDir[::-1])
        return nomeDir

    #mv
    def moverItens(self, nome, destino):
        root = self.aux
        if nome not in root.filhos:
            print("item não encontrado no diretório atual")
            return
        
        inode = root.filhos.pop(nome)

        destinoSplit = destino.split("/")
        if len(destinoSplit) > 1 and destino != "/":
            root = self.pathAbsoluto(destino)

        if self.aux.nome == "/" and (destino == ".." or destinoSplit[-1] == ".."):
            print(f"não é possível realizar tal ação")
            return
        
        if destino == "/":
            root = self.root
        elif destinoSplit[-1] == "..":
            root = root.pai
        elif destinoSplit[-1] == ".":
            pass
        else:
            root = root.filhos[destinoSplit[-1]]
        
        if root:
            #item já existe no diretório especificado, deseja substituir? (apagar o diretório com o mesmo nome e substituir)
            if nome in root.filhos:
                print("item já existe no diretório especificado")
                return
            inode.pai = root
            root.filhos[nome] = inode

In [2]:
arqSys = arquivos()

In [3]:
#TESTE: criando arquivos e diretórios na pasta root /
arqSys.criaArquivo("startup.log")
arqSys.criaDiretorio("program files")
arqSys.criaDiretorio("program files (x86)")
arqSys.criaDiretorio("users")
arqSys.listarItensDirAtual()


Diretório /

	ARQ  startup.log
	DIR  program files
	DIR  program files (x86)
	DIR  users



In [4]:
#TESTE: tentar criar os itens com mesmo nome dos já existentes
arqSys.criaArquivo("startup.log")
arqSys.criaDiretorio("program files")
arqSys.criaDiretorio("program files (x86)")
arqSys.criaDiretorio("users")

arquivo com nome 'startup.log' já existente no diretório atual
diretório com nome 'program files' já existente no diretório atual
diretório com nome 'program files (x86)' já existente no diretório atual
diretório com nome 'users' já existente no diretório atual


In [5]:
#TESTE: acessar um diretório a partir do caminho relativo e criar umas pastas nele
arqSys.moverParaDiretorio("program files")
arqSys.listarItensDirAtual()

arqSys.criaDiretorio("IIS")
arqSys.criaDiretorio("Java")
arqSys.criaDiretorio("dotnet")

arqSys.listarItensDirAtual()


Diretório /program files



Diretório /program files

	DIR  IIS
	DIR  Java
	DIR  dotnet



In [6]:
#TESTE: voltar para o diretório root e ir para o diretório IIS e criar 2 arquivos
arqSys.moverParaDiretorio("/")
arqSys.listarItensDirAtual()

arqSys.moverParaDiretorio("program files/IIS")
arqSys.criaArquivo("Microsoft.Web.Deployment.dll")
arqSys.criaArquivo("Microsoft.Web.Deployment.Tracing.dll")
arqSys.listarItensDirAtual()


Diretório /

	ARQ  startup.log
	DIR  program files
	DIR  program files (x86)
	DIR  users


Diretório /program files/IIS

	ARQ  Microsoft.Web.Deployment.dll
	ARQ  Microsoft.Web.Deployment.Tracing.dll



In [7]:
#TESTE: voltar para o diretório anterior e listar seus arquivos
arqSys.moverParaDiretorio("..")
arqSys.listarItensDirAtual()


Diretório /program files

	DIR  IIS
	DIR  Java
	DIR  dotnet



In [8]:
#TESTE: mover arquivos entre diretórios
arqSys.moverParaDiretorio("program files/IIS")
arqSys.listarItensDirAtual()
arqSys.moverParaDiretorio(".")
arqSys.listarItensDirAtual()

#arquivo sai do caminho /program files/IIS e vai para o root /
arqSys.moverItens("Microsoft.Web.Deployment.dll", "/")
arqSys.listarItensDirAtual()

arqSys.moverParaDiretorio("/")
arqSys.listarItensDirAtual()


Diretório /program files/IIS

	ARQ  Microsoft.Web.Deployment.dll
	ARQ  Microsoft.Web.Deployment.Tracing.dll


Diretório /program files/IIS

	ARQ  Microsoft.Web.Deployment.dll
	ARQ  Microsoft.Web.Deployment.Tracing.dll


Diretório /program files/IIS

	ARQ  Microsoft.Web.Deployment.Tracing.dll


Diretório /

	ARQ  startup.log
	DIR  program files
	DIR  program files (x86)
	DIR  users
	ARQ  Microsoft.Web.Deployment.dll



In [9]:
#TESTE: move o arquivo para o mesmo diretório
arqSys.moverParaDiretorio("/")
arqSys.moverItens("Microsoft.Web.Deployment.dll", ".")
arqSys.listarItensDirAtual()


Diretório /

	ARQ  startup.log
	DIR  program files
	DIR  program files (x86)
	DIR  users
	ARQ  Microsoft.Web.Deployment.dll



In [10]:
#TESTE: mover um arquivo usando path absoluto
arqSys.moverParaDiretorio("/")
arqSys.listarItensDirAtual()
arqSys.moverItens("Microsoft.Web.Deployment.dll", "/program files/Java")
arqSys.moverParaDiretorio("/program files/Java")
arqSys.listarItensDirAtual()


Diretório /

	ARQ  startup.log
	DIR  program files
	DIR  program files (x86)
	DIR  users
	ARQ  Microsoft.Web.Deployment.dll


Diretório /program files/Java

	ARQ  Microsoft.Web.Deployment.dll



In [11]:
#TESTE: mover diretórios
arqSys.moverParaDiretorio("/")
arqSys.listarItensDirAtual()
arqSys.moverItens("program files", "users")
arqSys.listarItensDirAtual()


arqSys.moverParaDiretorio("users")
arqSys.listarItensDirAtual()

arqSys.moverParaDiretorio("program files")
arqSys.listarItensDirAtual()


Diretório /

	ARQ  startup.log
	DIR  program files
	DIR  program files (x86)
	DIR  users


Diretório /

	ARQ  startup.log
	DIR  program files (x86)
	DIR  users


Diretório /users

	DIR  program files


Diretório /users/program files

	DIR  IIS
	DIR  Java
	DIR  dotnet



In [12]:
#TESTE: inserir dados em um arquivo e lê-los
arqSys.moverParaDiretorio("/users/program files/IIS")
arqSys.escreverNoArquivo("Microsoft.Web.Deployment.Tracing.dll", "4.668 5.88 3.5 8 3.5c2.12 0 3.879 1.168 5.168")
arqSys.escreverNoArquivo("Microsoft.Web.Deployment.Tracing.dll", "M16 8s-3-5.5-8-5.5S0 8 0 8s3 5.5 8 5.5S16 8 16")
arqSys.escreverNoArquivo("Microsoft.Web.Deployment.Tracing.dll", "8zM1.173 8a13.133 13.133 0 0 1 1.66-2.043C4.12")
arqSys.lerArquivo("Microsoft.Web.Deployment.Tracing.dll")


Lendo arquivo 'Microsoft.Web.Deployment.Tracing.dll'

	 4.668 5.88 3.5 8 3.5c2.12 0 3.879 1.168 5.168
	 M16 8s-3-5.5-8-5.5S0 8 0 8s3 5.5 8 5.5S16 8 16
	 8zM1.173 8a13.133 13.133 0 0 1 1.66-2.043C4.12

	tamanho do arquivo: 137


In [13]:
#TESTE: tentar apagar diretórios com itens
arqSys.moverParaDiretorio("/")
arqSys.listarItensDirAtual()
arqSys.removerItens("/users")
arqSys.listarItensDirAtual()


Diretório /

	ARQ  startup.log
	DIR  program files (x86)
	DIR  users

Não é possível apagar diretórios com itens dentro

Diretório /

	ARQ  startup.log
	DIR  program files (x86)
	DIR  users



In [14]:
#TESTE: tentar apagar diretórios com itens usando force
arqSys.moverParaDiretorio("/")
arqSys.listarItensDirAtual()
arqSys.removerItens("/users", "-f")
arqSys.removerItens("startup.log")
arqSys.listarItensDirAtual()


Diretório /

	ARQ  startup.log
	DIR  program files (x86)
	DIR  users


Diretório /

	DIR  program files (x86)

